In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("/kaggle/input/sloan-digital-sky-survey/Skyserver_SQL2_27_2018 6_51_39 PM.csv")
data.head()

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(data=data,x="class")

In [ ]:
sns.countplot(data=data,x="rerun") 

Interesting maybe rerun have only 1 value or not we have to examine it. 

In [ ]:
data["rerun"].groupby(data["rerun"]).count()

Okay we delete rerun column

In [ ]:
del data["rerun"]

In [ ]:
data.head()

In [ ]:
data["run"].groupby(data["run"]).count()

Run has values can't like rerun

In [ ]:
data["camcol"].groupby(data["camcol"]).count()

Camcol has also some value.

In [ ]:
data.describe()

Also 0 missing values is occur in this dataset.

In [ ]:
data.shape

Okay first we examine LinearSVM with all features.

In [ ]:
y=data["class"]

In [ ]:
x=data[data.columns.difference(['class'])]

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel="linear")
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_scale=sc.fit_transform(x)
from sklearn.model_selection import cross_val_score
cross_val_score(svm,x_scale,y,cv=5)

İt's worked excellent. This Dataset is linearly separable.

1 more example with train test split. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_scale, y, test_size=0.35, random_state=142)
svm=SVC(kernel="linear")
svm.fit(X_train,y_train)
ypred=ypred=svm.predict(X_test)
import sklearn.metrics as metr
print(metr.confusion_matrix(y_pred=ypred,y_true=y_test))
print(metr.accuracy_score(y_pred=ypred,y_true=y_test))
print(metr.classification_report(y_pred=ypred,y_true=y_test))

İt's worked just fine. 

Now We visualize these Dataset with T-SNE

In [ ]:
from sklearn.manifold import TSNE
x_visu=TSNE(n_components=2).fit_transform(x_scale)

In [ ]:
x_visuframe=pd.DataFrame(x_visu)
x_visuframe.columns=["bir","iki"]
x_visuframe.head()

In [ ]:
sns.relplot(data=x_visuframe,x="bir",y="iki",hue=y)

Can't work as expected. Howewer some island is very clear. 

Now We see LDA's performance.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
x_scale_lda=lda.fit_transform(x_scale,y)

In [ ]:
x_visuframelda=pd.DataFrame(x_scale_lda)
x_visuframelda.columns=["bir","iki"]
x_visuframelda.head()

In [ ]:
sns.relplot(data=x_visuframelda,x="bir",y="iki",hue=y)

It looks QSO and STARS is slearly separable and Galaxy is beetween these 2. İt's interesting. 

What if we can't use scalar:

In [ ]:
lda = LinearDiscriminantAnalysis()
x_lda=lda.fit_transform(x,y)
x_ldaframe=pd.DataFrame(x_lda)
x_ldaframe.columns=["bir","iki"]
sns.relplot(data=x_ldaframe,x="bir",y="iki",hue=y)

Can't see any difference between unscaled version. 

Maybe we delete CoLinear Variable and work just fine. 

In [ ]:
correlation=x.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(correlation,annot=True)

g and i and r and z is colinear.

delete i and r columns  and I thing it's solved.

In [ ]:
xnew=x[x.columns.difference(['i','r'])]
xnew.head()

In [ ]:
x.columns

In [ ]:
correlation=xnew.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(correlation,annot=True)

also delete mjd and g. it's the final result.

In [ ]:
xnew2=x[x.columns.difference(['mjd','g'])]
xnew2.head()

In [ ]:
lda = LinearDiscriminantAnalysis()
x_lda=lda.fit_transform(xnew2,y)
x_ldaframe=pd.DataFrame(x_lda)
x_ldaframe.columns=["bir","iki"]
sns.relplot(data=x_ldaframe,x="bir",y="iki",hue=y)

In [ ]:
xnew2

In [ ]:
correlation=xnew2.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(correlation,annot=True)

Okay lastly delete z and i 

In [ ]:
xnew3=x[x.columns.difference(["z","i"])]
xnew3.head()

In [ ]:
lda = LinearDiscriminantAnalysis()
x_lda=lda.fit_transform(xnew3,y)
x_ldaframe=pd.DataFrame(x_lda)
x_ldaframe.columns=["bir","iki"]
sns.relplot(data=x_ldaframe,x="bir",y="iki",hue=y)

In [ ]:
correlation=xnew3.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(correlation,annot=True)

Last Last delete g and r

In [ ]:
xnew4=xnew3[x.columns.difference(["z","i"])]
xnew3.head()

In [ ]:
correlation=xnew4.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(correlation,annot=True)

In [ ]:
xnew5=xnew4[xnew4.columns.difference(["mjd","r","plate"])]
correlation=xnew5.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(correlation,annot=True)

Okay it's just fine. This is last 

In [ ]:
lda = LinearDiscriminantAnalysis()
x_lda=lda.fit_transform(xnew5,y)
x_ldaframe=pd.DataFrame(x_lda)
x_ldaframe.columns=["bir","iki"]
sns.relplot(data=x_ldaframe,x="bir",y="iki",hue=y)

### This data is this. :) :)